In [ ]:
## Task 1

```
# import warnings
# warnings.filterwarnings("ignore")

# import numpy as np
# import pandas as pd
# import io
# import requests

# df_url = 'https://raw.githubusercontent.com/akmand/datasets/main/california_housing.csv'
# url_content = requests.get(df_url, verify=False).content
# data1 = pd.read_csv(io.StringIO(url_content.decode('utf-8')))
# ```


# Follow the steps below and show the full code.
# - Use data1.sample(105, random_state=56)  to take sample  
# - Select [[longitude,latitude,housing_median_age,total_rooms,total_bedrooms,ocean_proximity] ] as feature and [median_house_value] as target                    
# - Fill missing values  with mean of that column  
# - Encode the categorical feature(s)       
# - Split the data with test size =13% and random_state = 12         
# - Create Decision tree model with max_depth = 10
# - Calculate test and train accuracy


#1st step: Import libraries
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import io
import requests
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

df_url = 'https://raw.githubusercontent.com/akmand/datasets/main/california_housing.csv'
url_content = requests.get(df_url, verify=False).content
data1 = pd.read_csv(io.StringIO(url_content.decode('utf-8')))

sample1 = data1.sample(105, random_state=56)
sample1.isna().sum()
sample1['total_bedrooms'] = sample1['total_bedrooms'].fillna(sample1['total_bedrooms']).mean()
sample1.isna().sum()
sample1.head()
sample1_ohe = pd.get_dummies(sample1, columns=['ocean_proximity']) # use one-hot-encoding to replaced categorical column
X = sample1_ohe.drop('median_house_value', axis=1)
y = sample1_ohe['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.13, random_state=12)
dt = DecisionTreeRegressor(max_depth=10, random_state=12)
dt.fit(X_train, y_train)
dt.score(X_train, y_train)
dt.score(X_test, y_test)


In [ ]:
## Task 2

# **Use Data1 given Task 1**

# Follow the steps below and show the full code.

# - Use data1.sample(500, random_state=56) to take sample
# - Select [[longitude,latitude,housing_median_age,total_rooms,total_bedrooms,ocean_proximity]] as feature and [median_house_value] as target
# - Remove missing values with mean of that column
# - Encode Categorical feaures
# - Split the data with test size =13% and random_state = 12
# - Create regression model
# - Calculate test and train accuracy
# - Which feature variable exhibiting the highest absolute influence on the model’s output


sample2 = data1.sample(500, random_state=56)
sample2.isna().sum()
sample2['total_bedrooms'] = sample2['total_bedrooms'].fillna(sample2['total_bedrooms']).mean()
sample2.isna().sum()
sample2.head()
sample2_ohe = pd.get_dummies(sample2, columns=['ocean_proximity'])
X = sample2_ohe.drop('median_house_value', axis=1)
y = sample2_ohe['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.13, random_state=12)
dtr2 = DecisionTreeRegressor(max_depth=10, random_state=12)
dtr2.fit(X_train, y_train)
print(f"Train accuracy: {dtr2.score(X_train, y_train): .4f}")
print(f"Test accuracy: {dtr2.score(X_test, y_test): .4f}")
dtr2.feature_importances_
dtr2.feature_names_in_
pd.Series(data = dtr2.feature_importances_, index=dtr2.feature_names_in_).sort_values(ascending=False)

In [ ]:
# Task 4

from sklearn.cluster import DBSCAN

df_url = 'https://raw.githubusercontent.com/aishwarya13-r/DAB200/refs/heads/main/circular_clusters_with_noise.csv'
url_content = requests.get(df_url, verify=False).content
data2 = pd.read_csv(io.StringIO(url_content.decode('utf-8')))



# - Visualize the data using a scatter plot (x, y) using code below


# - Apply DBSCAN clustering and identify core
# - Count how many points fall into noise vs clusters.


data2.head()
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.scatter(data2['x'], data2['y'], s=10, color='gray')
plt.title('Original Data: Circular Clusters with Noise')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

X = data2[['x','y']]


dbscan = DBSCAN(eps=0.2, min_samples=5)
clusters = dbscan.fit_predict(X)
data2['cluster']
plt.figure(figsize=(8, 6))
unique_labels = np.unique(data2['cluster'])
colors = {-1: 'red', 0: 'blue', 1: 'green', 2: 'purple'}  # Extend as needed

for label in unique_labels:
    mask = data2['cluster'] == label
    cluster_name = "Noise" if label == -1 else f"Cluster {label}"
    plt.scatter(data2[mask]['x'], data2[mask]['y'],
                c=colors.get(label, 'gray'), s=10, label=cluster_name)

plt.title("DBSCAN Clustering with Core and Border Points")
plt.legend(loc='upper left')
plt.show()
n_clusters = len(set(data2['cluster'])) - (1 if -1 in data2['cluster'].values else 0)
n_noise = np.sum(data2['cluster'] == -1)

print(f"Number of clusters: {n_clusters}")
print(f"Number of noise: {n_noise}")

In [ ]:
# Task 5
# - Implement Agglomerative Clustering for data 2 given in task 4


from sklearn.cluster import AgglomerativeClustering
agg = AgglomerativeClustering().fit(X)
ac_predictions=agg.fit_predict(X)
ac_predictions
agg.labels_


# Option 1: Use Dendrogram

import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt

Z = sch.linkage(X, method='ward')  # You can try other linkage methods like 'average', 'single', etc.

plt.figure(figsize=(10, 7))
sch.dendrogram(Z)
plt.title("Dendrogram")
plt.xlabel("Samples")
plt.ylabel("Distance")
plt.show()


# Option 2: Use Silhouette Score

# The silhouette score measures how well each sample fits within its cluster. A higher silhouette score indicates that the clusters are well-defined.

# Silhouette score ranges from -1 to +1.

# - A score close to +1 indicates that the samples are well clustered.
# - A score close to 0 indicates overlapping clusters.
# - A negative score suggests that samples might have been assigned to the wrong clusters.


#Use silhouette score to find the best number of clusters:
from sklearn.metrics import silhouette_score
sil_score = silhouette_score(X, ac_predictions)
print(f"Silhouette Score: {sil_score}")

In [ ]:
# Task 6: Part I

# - Implement K-means clustering for data used in task 4


from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)  # Set a random_state for reproducibility
clusters2 = kmeans.fit(data2)  # Fit the KMeans model to the data

clusters2.predict(data2)
kmeans.cluster_centers_.shape


# Assuming you have already fitted the model with kmeans.fit(data2)
labels = kmeans.labels_  # Cluster labels for each data point
centroids = kmeans.cluster_centers_  # Coordinates of the cluster centroids

# Step 1: Plot the data points, color-coded by cluster labels
plt.scatter(data2.iloc[:, 0], data2.iloc[:, 1], c=labels, cmap='viridis', alpha=0.6)

# Step 2: Plot the cluster centers as red 'X' marks
plt.scatter(centroids[:, 0], centroids[:, 1], s=200, c='red', marker='X', label="Centroids")

# Add title, labels, and legend
plt.title('KMeans Clustering')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()

# Show the plot
plt.show()


# 1. DBSCAN clusters by density, not distance to a centroid:  
#     - DBSCAN grows clusters by "chaining" together points that lie in high density
# 2. No assumption of COnvex or Spherical shape: 
#     - KMeans optimizes around a single center and creates convex (ball-shaped) clusters. 
#     - DBSCAN links nearby points, regardless of overall shape, so it follws the curving geometry of each ring
# 3. Automaticaly detects number of clusters: 
#     - DBSCAN discovers number of clusters based on density thresholds.

In [ ]:
# Task 6: Part II

# Based on the hierarchical clustering dendrogram generated using below code, if you decide to form 3 distinct clusters, identify and list the data points assigned to each of the resulting clusters


from scipy.cluster.hierarchy import dendrogram, linkage
link = linkage(data[:10,:], method='ward')used

plt.figure(figsize=(12, 6))
dendrogram(link,
           orientation='top',
           distance_sort='ascending',
           show_leaf_counts=False,
           truncate_mode='level',  # limit the depth of the tree
           p=20)  # show only last 20 merges for readability
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Sample Index')
plt.show()


# Select only 'x' and 'y' for clustering
X = data2[['x', 'y']].values

link = linkage(X[:15,:], method='ward')  # X - data used

plt.figure(figsize=(12, 6))
dendrogram(link,
           orientation='top',
           distance_sort='ascending',
           show_leaf_counts=False,
           truncate_mode='level')
plt.title('Hierarchical Clustering Dendrogram')
plt.show()